In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/port_irimare_no_locations.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


The year is 1925, under a dark moonless night, you are riding on a bus towards Port Irimare with two other passengers. The road is flooded by a thick fog, yet, the old man behind the wheel zip through the winding road as if he had done this a thousand times. 
The danger worries you slightly but you can’t help enjoying the thrill of it. Suddenly, you felt the old man slam on the brakes. You jerked forward and back after you felt the entire vehicle collide with something. 
The sound of the crash was loud, and the screeching noise of metal tearing through metal was just all too familiar, and that moment you knew the old man had collided with another vehicle. 
You and the other two passengers climbed out from the window of the wrecked bus and investigated the wreckage. 
The front of the bus had crumpled inwards and had completely crushed the old bus driver. What was once recognizable could only now be described as pounded flesh. 
The other vehicle was a police car, but there was nobody ins

I select option 2. TURN: 1 


As you stand before the fog-shrouded entrance of Port Irimare, you decide to introduce yourself to the group. You carefully craft your introduction to avoid any mention of your true purpose in this town. "I'm Jordan," you say